## Make predictions in production

In [ ]:
# Load the scaler, label encoders, and trained model
label_Encoder = joblib.load('../models/gr_label_enc.pkl')
scaler = joblib.load('../models/gr_scaler.pkl')
loaded_model = joblib.load('../models/gr_gradient_boosting_model.joblib')
print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
# Read DataFrame
df = pd.read_csv('../data/processed/auto_cons_us_processed.csv', sep=',', header=0)

# Show DataFrame
print(df.head(3))

   target  # of cylinders  Engine displacement  Engine power  Weight  \
0      13               8                    5         130.0    1589   
1      16               8                    6         165.0    1675   
2      13               8                    5         150.0    1559   

   Acceleration  Year Origin  
0          12.0  2005     US  
1          11.5  2005     US  
2          11.0  2005     US  


In [ ]:
# Prediction 1, the result is aproximately 16
prediction1 = {
'# of cylinders':8,
'Engine displacement':6,
'Engine power':160,
'Weight':1670,
'Acceleration':12,
'Year':2005,
'Origin': 'US'
}

In [ ]:
def preprocess_data(df):
    '''Function to preprocess new data for prediction'''
    # Apply the imported label_Encoder
    for col, encoder in label_Encoder.items():
        if col in df.select_dtypes(include=['object']).columns:
            try:
                df[col] = encoder.transform(df[col])
            except ValueError as e:
                raise ValueError(f"Error al transformar la columna '{col}'. "
        f"Asegúrate de que los valores en los nuevos datos coincidan con los datos de entrenamiento. "
        f"Detalles: {e}"
                )
    # Aplicar escalado usando el scaler guardado
    df = scaler.transform(df)
    return df

def make_prediction(single_case):
    # Convertir el caso único en DataFrame
    single_case_df = pd.DataFrame(single_case)
    # Preprocesar el caso único
    processed_single_case = preprocess_data(single_case_df)

    prediction = loaded_model.predict(processed_single_case)
    
    print(f'Predicción: {prediction[0]:.2f}')